## Part 3: Redux Toolkit - The Industry Standard 🏢

**Redux** has been the most popular state management solution for years. **Redux Toolkit (RTK)** is the modern, official way to use Redux with less boilerplate.

### Why Learn Redux?

- 🏢 **Industry standard** - Used by most large companies (including Nigerian tech companies like Andela, Kuda)
- 📚 **Large ecosystem** - Lots of middleware, tools, and community support
- 🎯 **Strict patterns** - Predictable state updates

### Installation

```bash
npm install @reduxjs/toolkit react-redux
```

---

### 3.1 Redux Core Concepts

Before coding, understand these terms:

| Concept | Description | Example |
|---------|-------------|--------|
| **Store** | The single source of truth | All your app's state lives here |
| **Slice** | A piece of state + its reducers | `chatSlice`, `authSlice` |
| **Action** | An event that describes what happened | `{ type: 'chat/addMessage', payload: {...} }` |
| **Reducer** | A function that updates state based on action | Takes state + action, returns new state |
| **Dispatch** | The function to send actions | `dispatch(addMessage({...}))` |
| **Selector** | A function to read state | `(state) => state.chat.messages` |


## 3.1 Redux Core Concepts

Before writing any Redux code, you must understand these core ideas. Redux is simply a **predictable way to manage global state** in large applications.

---

### 🗄️ Store

**Description:**
The **store** is the single source of truth. It holds the **entire global state** of your application.

* Your app has **one store**
* All slices live inside the store

**Example (mental model):**

```js
{
  auth: { user: null, isLoggedIn: false },
  chat: { messages: [] }
}
```

---

### 🧩 Slice

**Description:**
A **slice** is a feature-based section of the store **plus** the reducers that manage it.

Redux Toolkit encourages splitting your app by features.

**Examples:**

* `authSlice`
* `chatSlice`
* `settingsSlice`

Each slice controls **only its own part of the state**.

---

### 📨 Action

**Description:**
An **action** is an object that describes **what happened** in the application.

* Must have a `type`
* Can include data via `payload`
* Does **not** change state directly

**Example:**

```js
{
  type: "chat/addMessage",
  payload: { text: "Hello world" }
}
```

Think of actions as **event messages**.

---

### 🔁 Reducer

**Description:**
A **reducer** is a function that decides **how state should change** in response to an action.

* Takes `state` + `action`
* Returns the new state
* Must be predictable

**Example:**

```js
addMessage: (state, action) => {
  state.messages.push(action.payload);
}
```

Reducers answer the question:
👉 *“Given this action, how should the state update?”*

---

### 🚀 Dispatch

**Description:**
`dispatch` is the function used to **send an action** to the store.

**Flow:**

```
Component → dispatch(action) → reducer → store updates → UI re-renders
```

**Example:**

```js
dispatch(addMessage({ text: "Hi!" }))
```

No dispatch = no state change.

---

### 🔍 Selector

**Description:**
A **selector** is a function that reads data from the Redux store.

Selectors:

* Keep components clean
* Improve performance
* Hide store structure

**Example:**

```js
(state) => state.chat.messages
```

Used in React with:

```js
useSelector(selectMessages)
```

### Redux Data Flow:

```
User clicks "Send" 
    → dispatch(addMessage(msg))
    → Reducer receives action
    → Reducer updates state
    → Component re-renders with new state
```

---

## 🧠 Summary

Redux works by:

1. **Dispatching actions**
2. **Reducers updating slices**
3. **Store holding the updated state**
4. **UI reacting to changes**

Once this flow is clear, Redux becomes **very easy**.


### 3.2 Creating a Slice

A **slice** contains the reducer logic and actions for a specific feature:

```javascript
// store/slices/chatSlice.js
import { createSlice } from '@reduxjs/toolkit'

const initialState = {
  messages: [],
  isAgentTyping: false,
  error: null
}

const chatSlice = createSlice({
  name: 'chat',
  initialState,
  reducers: {
    addMessage: (state, action) => {
      // RTK uses Immer - you can "mutate" state directly!
      state.messages.push(action.payload)
    },
    
    setAgentTyping: (state, action) => {
      state.isAgentTyping = action.payload
    },
    
    setError: (state, action) => {
      state.error = action.payload
    },
    
    clearMessages: (state) => {
      state.messages = []
      state.error = null
    }
  }
})

// Export actions
export const { addMessage, setAgentTyping, setError, clearMessages } = chatSlice.actions

// Export reducer
export default chatSlice.reducer
```

### Key Points:

1. **`createSlice`** - Generates action creators and reducers automatically
2. **Immer integration** - You can write "mutating" code, RTK handles immutability
3. **`action.payload`** - The data passed with the action

---

### 3.3 Configuring the Store

```javascript
// store/index.js
import { configureStore } from '@reduxjs/toolkit'
import chatReducer from './slices/chatSlice'
import authReducer from './slices/authSlice'

const store = configureStore({
  reducer: {
    chat: chatReducer,
    auth: authReducer
  }
})

export default store
```

### Providing the Store to React

```jsx
// main.jsx
import React from 'react'
import ReactDOM from 'react-dom/client'
import { Provider } from 'react-redux'
import store from './store'
import App from './App'

ReactDOM.createRoot(document.getElementById('root')).render(
  <React.StrictMode>
    <Provider store={store}>
      <App />
    </Provider>
  </React.StrictMode>
)
```

**Note:** Unlike Zustand, Redux requires a Provider wrapper!

---

### 3.4 Using Redux in Components

```jsx
// components/ChatWindow.jsx
import { useSelector } from 'react-redux'

function ChatWindow() {
  // useSelector to read state
  const messages = useSelector((state) => state.chat.messages)
  const isAgentTyping = useSelector((state) => state.chat.isAgentTyping)
  
  return (
    <div className="chat-window">
      {messages.map((msg, index) => (
        <div key={index} className={`message ${msg.role}`}>
          <strong>{msg.role === 'user' ? 'You' : 'AI Agent'}:</strong>
          <p>{msg.content}</p>
        </div>
      ))}
      
      {isAgentTyping && (
        <div className="typing-indicator">AI is thinking...</div>
      )}
    </div>
  )
}

export default ChatWindow
```

---

### 3.5 Dispatching Actions

```jsx
// components/ChatInput.jsx
import { useState } from 'react'
import { useDispatch } from 'react-redux'
import { addMessage, setAgentTyping } from '../store/slices/chatSlice'

function ChatInput() {
  const [input, setInput] = useState('')
  const dispatch = useDispatch()
  
  const handleSubmit = async (e) => {
    e.preventDefault()
    if (!input.trim()) return
    
    // Dispatch actions
    dispatch(addMessage({ role: 'user', content: input }))
    setInput('')
    
    dispatch(setAgentTyping(true))
    
    // Simulate API call
    setTimeout(() => {
      dispatch(addMessage({ 
        role: 'agent', 
        content: 'I received your message!' 
      }))
      dispatch(setAgentTyping(false))
    }, 1500)
  }
  
  return (
    <form onSubmit={handleSubmit}>
      <input
        value={input}
        onChange={(e) => setInput(e.target.value)}
        placeholder="Type a message..."
      />
      <button type="submit">Send</button>
    </form>
  )
}

export default ChatInput
```

---

### 3.6 Redux: Async Actions with createAsyncThunk

For API calls, use `createAsyncThunk`:

```javascript
// store/slices/chatSlice.js
import { createSlice, createAsyncThunk } from '@reduxjs/toolkit'

// Async thunk for sending messages to AI
export const sendMessageToAgent = createAsyncThunk(
  'chat/sendMessageToAgent',
  async (userMessage, { rejectWithValue }) => {
    try {
      const response = await fetch('http://localhost:8000/api/chat', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ message: userMessage })
      })
      
      if (!response.ok) {
        throw new Error('Failed to get response')
      }
      
      const data = await response.json()
      return data
      
    } catch (error) {
      return rejectWithValue(error.message)
    }
  }
)

const chatSlice = createSlice({
  name: 'chat',
  initialState: {
    messages: [],
    isAgentTyping: false,
    error: null
  },
  reducers: {
    addMessage: (state, action) => {
      state.messages.push(action.payload)
    },
    clearMessages: (state) => {
      state.messages = []
    }
  }
})

export const { addMessage, clearMessages } = chatSlice.actions
export default chatSlice.reducer
```

### Using Async Thunk in Component:

```jsx
import { useDispatch } from 'react-redux'
import { addMessage, sendMessageToAgent } from '../store/slices/chatSlice'

function ChatInput() {
  const dispatch = useDispatch()
  
  const handleSubmit = (e) => {
    e.preventDefault()
    
    // Add user message
    dispatch(addMessage({ role: 'user', content: input }))
    
    // Send to AI (handles loading/error states automatically)
    dispatch(sendMessageToAgent(input))
  }
  // ...
}
```

---

## Part 4: Zustand vs Redux - Comparison

| Feature | Zustand | Redux Toolkit |
|---------|---------|---------------|
| **Bundle size** | ~1KB | ~10KB |
| **Boilerplate** | Minimal | Moderate (but much less than old Redux) |
| **Learning curve** | Easy | Steeper |
| **Provider required** | No | Yes |
| **Async handling** | Native async/await | createAsyncThunk |
| **TypeScript** | Excellent | Excellent |
| **Community/Jobs** | Growing | Massive |
| **Best for** | Small-medium apps | Large enterprise apps |

### When to Use What?

**Choose Zustand when:**
- Building a small to medium application
- You want minimal boilerplate
- You're the only developer or small team
- You want to get started quickly

**Choose Redux when:**
- Building a large enterprise application
- Working in a team that already knows Redux
- You need extensive middleware ecosystem
- You want strict patterns and best practices enforced
- Job requirements specify Redux

---

## 📝 Practice Exercises

### Exercise 1: Basic Zustand Store (Beginner)

Create a Zustand store for a **Nigerian Naira Expense Tracker** with:

**State:**
- `expenses` - Array of expense objects `{ id, description, amount, category, date }`
- `totalBudget` - Monthly budget in Naira (e.g., ₦500,000)

**Actions:**
- `addExpense(expense)` - Add a new expense
- `removeExpense(id)` - Remove an expense by ID
- `setBudget(amount)` - Update the monthly budget

**Computed:**
- `getTotalSpent()` - Calculate total amount spent
- `getRemainingBudget()` - Calculate remaining budget

**Bonus:** Add persist middleware to save expenses to localStorage.

---

### Exercise 2: Redux Toolkit Slice (Intermediate)

Convert the expense tracker to Redux Toolkit:

1. Create an `expenseSlice` with the same state and reducers
2. Configure the store
3. Create selectors for `selectTotalSpent` and `selectRemainingBudget`
4. Build a simple UI that displays expenses and allows adding new ones

---

### Exercise 3: Multi-Store Application (Advanced)

Build a complete state management system for a **"Naija AI Document Assistant"** with:

**Stores/Slices needed:**

1. **authStore** - User authentication
   - `user`, `isAuthenticated`, `token`
   - `login()`, `logout()`, `refreshToken()`

2. **documentStore** - Document management
   - `documents`, `selectedDocument`, `isUploading`
   - `uploadDocument()`, `deleteDocument()`, `selectDocument()`

3. **chatStore** - AI chat functionality
   - `messages`, `isAgentTyping`
   - `sendQuery()`, `clearChat()`

4. **uiStore** - UI state
   - `sidebarOpen`, `theme`, `notifications`
   - `toggleSidebar()`, `setTheme()`, `addNotification()`

**Requirements:**
- When a document is uploaded, add a notification
- When user logs out, clear chat and documents
- Persist theme preference

---

## 🎯 Summary

### What You Learned:

1. **Why external state management matters** for complex AI applications

2. **Zustand fundamentals:**
   - Creating stores with `create()`
   - Selective subscriptions for performance
   - Async actions with native async/await
   - Persist and DevTools middleware

3. **Redux Toolkit fundamentals:**
   - Creating slices with `createSlice()`
   - Configuring the store
   - Using `useSelector` and `useDispatch`
   - Async thunks with `createAsyncThunk`

4. **When to use which:**
   - Zustand for simplicity and small-medium apps
   - Redux for enterprise apps and large teams

---

*Happy coding! 🇳🇬*